In [ ]:
import airbyte as ab

In [ ]:
ab.get_available_connectors()

In [ ]:
ACCESS_TOKEN = {
                "loginCustomerId": "<CUSTOMER_ID>",
                "setToExpireOn": "2024-07-19T14:45:13.420Z",
                "token": "<TOKEN>",
                "clientSecret": "<CLIENT_SECRET>",
                "clientId": "<CLIENT_ID>",
                "refreshToken": "<REFRESH_TOKEN>",
                "developerToken": "<DEVELOPER_TOKEN>"
            }

In [ ]:
import requests

TMS_FIELD_TO_CONFIG_VARIABLE_MAPPER = {
    "login_customer_id": "loginCustomerId",
    "set_to_expire_on": "setToExpireOn",
    "access_token": "token",
    "client_secret": "clientSecret",
    "client_id": "clientId",
    "refresh_token": "refreshToken",
    "developer_token": "developerToken"
}


class CreateGoogleAdsSourceConfiguration:
    CREDENTIAL_FORMAT = {
        "developer_token": {
            "type": "string",
            "required": True
        },
        "client_id": {
            "type": "string",
            "required": True
        },
        "client_secret": {
            "type": "string",
            "required": True
        },
        "refresh_token": {
            "type": "string",
            "required": True
        },
        "access_token": {
            "type": "string",
            "required": False
        }
    }

    CONFIGURATION_FORMAT = {
        "sourceType": {
            "type": "string",
            "required": True
        },
        "credentials": {
            "type": "object",
            "required": True
        },
        "customer_status_filter": {
            "type": "array",
            "required": False
        },
        "customer_id": {
            "type": "string",
            "required": False
        },
        "start_date": {
            "type": "string",
            "required": False
        },
        "end_date": {
            "type": "string",
            "required": False
        },
        "conversion_window_days": {
            "type": "integer",
            "required": False,
            "default": 14
        }
    }

    PAYLOAD_FORMAT = {
        "name": {
            "type": "string",
            "required": True
        },
        "workspaceId": {
            "type": "string",
            "required": True
        },
        "configuration": {
            "type": "object",
            "required": True
        }
    }

    def __init__(self, tms_token: dict, customer_id: str, start_date: str, end_date: str, conversion_window_days: int,
                 *args, **kwargs) -> None:
        self.sourceType = "google-ads"
        self.credentials = self.create_credentials(tms_token)
        self.customer_status_filter = None
        self.customer_id = customer_id
        self.start_date = start_date
        self.end_date = end_date
        self.conversion_window_days = conversion_window_days

    def create_credentials(self, tms_token: dict) -> dict:
        credentials = {}
        for field in CreateGoogleAdsSourceConfiguration.CREDENTIAL_FORMAT.keys():
            tms_value = tms_token.get(TMS_FIELD_TO_CONFIG_VARIABLE_MAPPER.get(field))
            if tms_value:
                credentials[field] = tms_value
            else:
                if CreateGoogleAdsSourceConfiguration.CREDENTIAL_FORMAT[field].get("required"):
                    raise ValueError(f"Missing required field: {field}")

        return credentials

    def create_configuration_payload(self) -> dict:
        configuration = {}

        for field in CreateGoogleAdsSourceConfiguration.CONFIGURATION_FORMAT.keys():
            field_value = getattr(self, field)
            if field_value:
                configuration[field] = field_value
            else:
                if CreateGoogleAdsSourceConfiguration.CONFIGURATION_FORMAT[field].get("required"):
                    raise ValueError(f"Missing required field: {field}")

        return configuration

    def create_payload(self, name: str) -> dict:
        payload = {
            "name": name,
            "configuration": self.create_configuration_payload()
        }

        return payload

In [ ]:
google_payload = CreateGoogleAdsSourceConfiguration(ACCESS_TOKEN, "3858104332", "2024-05-01", "2024-05-31", 14).create_payload('source-google-microscope-may-2024')

In [ ]:
source_gads = ab.get_source('source-google-ads', install_if_missing=True, config=google_payload.get('configuration'))


In [ ]:
source_gads.get_available_streams()

In [ ]:
source_gads.check()

In [ ]:
source_gads.select_streams(['ad_group'])

In [ ]:

source_gads.get_stream_json_schema('ad_group')

In [ ]:
import time

start_time = time.time()
result = source_gads.read()


end_time = time.time()
# print(f"Time required : {end_time - start_time}")

In [ ]:
import time

start_time = time.time()
cache = ab.get_default_cache()
result = source_gads.read(cache=cache)


end_time = time.time()
print(f"Time required : {end_time - start_time}")

In [ ]:
source_gads.select_streams(['campaign'])

In [ ]:
source_gads.get_selected_streams()

In [ ]:
result = source_gads.read()

In [ ]:
#print result 
print(dir(result))

In [ ]:
print(len(result))

In [ ]:
df = result.cache.get_pandas_dataframe('ad_group')

In [ ]:
len(df)

In [ ]:
df.head()